In [12]:
import pandas as pd
import numpy as np

# ===== 1) 멀티헤더 읽기 =====
df = pd.read_csv("북숭아_개화_만발_15~25.csv", header=[0,1])

# ===== 2) 컬럼 정리 =====
df.columns = [
    (str(a).strip() if str(a) != "nan" else "") + 
    ("_" + str(b).strip() if ("Unnamed" not in str(b)) and (str(b) != "nan") else "")
    for a,b in df.columns
]

# 공백 제거
df.columns = [c.strip("_") for c in df.columns]

# ===== 3) 결측치 강력 처리 =====
def clean(x):
    x = str(x).strip()
    if x in ["", "결측", "관측 안됨", "—", "―", "-", "nan", "NaN", "None"]:
        return np.nan
    return x

df = df.applymap(clean)

# ===== 4) 완전 빈 행 제거 =====
df = df.dropna(how="all")  # NaN만 있을 경우 제거
df = df[~df.apply(lambda r: all(str(v).strip()=="" or str(v)=="nan" for v in r), axis=1)]

# ===== 5) 날짜 변환 =====
df["개화"] = pd.to_datetime(df["복숭아_개화"], errors="coerce")
df["만발"] = pd.to_datetime(df["복숭아_만발"], errors="coerce")

# ===== 6) 연도 숫자 변환 =====
df["연도"] = pd.to_numeric(df["년도"], errors="coerce")

# ===== 7) 차이 계산 =====
df["차이"] = (df["만발"] - df["개화"]).dt.days

# ===== 8) 지점별 평균 차이 ====
diff_mean = df.groupby("지점")["차이"].mean()

# ===== 9) 결측 보정 =====
def fill(row):
    if pd.isna(row["만발"]):
        avg = diff_mean.get(row["지점"], np.nan)
        if not pd.isna(avg) and not pd.isna(row["개화"]):
            return row["개화"] + pd.Timedelta(days=avg)
    return row["만발"]

df["만발"] = df.apply(fill, axis=1)

# ===== 저장 =====
df.to_csv("북숭아_개화_만발_보정.csv", index=False)


C:\Users\boy98\AppData\Local\Temp\ipykernel_28008\1260062779.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean)
